<a href="https://colab.research.google.com/github/ewidman/ct_project/blob/master/article_feature_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Good overview of ranking algos [here](https://academic.oup.com/bib/article/20/1/178/4091291)

[bayesian ranking system](https://towardsdatascience.com/bayesian-ranking-system-77818e63b57b)

could [auto-sklearn](https://automl.github.io/auto-sklearn/master/manual.html#manual) be useful here or [automated feature engineering](https://towardsdatascience.com/automated-feature-engineering-in-python-99baf11cc219)

[list of journal scores](https://instr.iastate.libguides.com/journaleval/rankings)

CiteScore data freely available [here](https://www.scopus.com/sources) and more info available [here](https://www.elsevier.com/authors/tools-and-resources/measuring-a-journals-impact)


In [ ]:
#All relevant installations
!pip install pandas==1.2.4
!pip install mc4
!pip install openpyxl==2.6.0
#!pip list
import pandas as pd
import mc4
import openpyxl
from google.colab import files
import sklearn
import io

#enable data table display for google colab
%load_ext google.colab.data_table

#disable data table display
#%unload_ext google.colab.data_table

In [2]:
print(pd.__version__)
print(openpyxl.__version__)

1.2.4
2.6.0


In [30]:
uploaded = files.upload()
cite_score = pd.read_csv(io.BytesIO(uploaded['citescore_2019.csv']))

Saving citescore_2019.csv to citescore_2019.csv


KeyError: ignored

In [ ]:
#do I need a seperate cell for each uploaded csv?
uploaded = files.upload()
asjc_code = pd.read_csv(io.BytesIO(uploaded['asjc_code.csv']))

In [28]:
cite_score.head()
#asjc_code[['asjc_code', 'description', 'major_grouping']]

,Scopus Source ID,Title,Citation Count,Scholarly Output,Percent Cited,CiteScore,SNIP,SJR,Scopus ASJC Code (Sub-subject Area),Scopus Sub-Subject Area,Percentile,RANK,Rank Out Of,Publisher,Type,Open Access,Quartile,Top 10% (CiteScore Percentile),URL Scopus Source ID,Print ISSN,E-ISSN
0,12128,Behavioural and Cognitive Psychotherapy,652,226,70.8,2.9,0.981,0.805,3203.0,Clinical Psychology,68.0,88.0,275.0,Cambridge University Press,j,NO,2.0,False,https://www.scopus.com/sourceid/12128,13524658,14691833
1,12366,Photogrammetric Engineering and Remote Sensing,644,260,60.8,2.5,0.708,0.521,1903.0,Computers in Earth Sciences,48.0,21.0,40.0,American Society for Photogrammetry and Remote...,j,NO,3.0,False,https://www.scopus.com/sourceid/12366,991112,NaN
2,12894,Chinese Journal of Chromatography (Se Pu),649,731,44.5,0.9,0.282,0.199,1500.0,General Chemical Engineering,29.0,200.0,281.0,Zhongguo Kexueyan Ganguang Huaxue Yanjiusuo,j,NO,3.0,False,https://www.scopus.com/sourceid/12894,10008713,18722059
3,12894,Chinese Journal of Chromatography (Se Pu),649,731,44.5,0.9,0.282,0.199,1603.0,Electrochemistry,17.0,31.0,37.0,Zhongguo Kexueyan Ganguang Huaxue Yanjiusuo,j,NO,4.0,False,https://www.scopus.com/sourceid/12894,10008713,18722059
4,12894,Chinese Journal of Chromatography (Se Pu),649,731,44.5,0.9,0.282,0.199,1602.0,Analytical Chemistry,15.0,101.0,119.0,Zhongguo Kexueyan Ganguang Huaxue Yanjiusuo,j,NO,4.0,False,https://www.scopus.com/sourceid/12894,10008713,18722059


In [ ]:
asjc_code=asjc_code[['asjc_code', 'description', 'major_grouping']]

In [22]:
asjc_code.columns

Index(['asjc_code', 'description', 'major_grouping', 'Unnamed: 3'], dtype='object')

In [4]:
cite_score.columns

Index(['Scopus Source ID', 'Title', 'Citation Count', 'Scholarly Output',
       'Percent Cited', 'CiteScore', 'SNIP', 'SJR',
       'Scopus ASJC Code (Sub-subject Area)', 'Scopus Sub-Subject Area',
       'Percentile', 'RANK', 'Rank Out Of', 'Publisher', 'Type', 'Open Access',
       'Quartile', 'Top 10% (CiteScore Percentile)', 'URL Scopus Source ID',
       'Print ISSN', 'E-ISSN'],
      dtype='object')

In [18]:
cite_score["Scopus ASJC Code (Sub-subject Area)"] == 2730
#Oncology = 2730







0        False
1        False
2        False
3        False
4        False
         ...  
57013    False
57014    False
57015    False
57016    False
57017    False
Name: Scopus ASJC Code (Sub-subject Area), Length: 57018, dtype: bool

In [ ]:
#import citescore data
from google.colab import drive
import pandas as pd
drive.mount('/content/gdrive')

#df = pd.read_csv("/content/gdrive/Shareddrive/Newsfeed_pipelines/Topics_pipeline/Research/cite_score.gsheet")

df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/cite_score2.gsheet")


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


OSError: ignored

In [ ]:
#import the article data (note...this is all scraped from external website. None of the data is proprietary)
from google.colab import files

uploaded = files.upload()
df = pd.read_csv(io.BytesIO(uploaded['G26_Neuro.csv']))
print(df.head)

Saving G26_Neuro.csv to G26_Neuro (2).csv
<bound method NDFrame.head of                TOPIC  ARTICLE_ID  ...  AVG_ALTMETRIC_SCORE  AVG_MEDICAL_DENSITY
0      G26/Neurology    28979831  ...            39.443029             0.928619
1      G26/Neurology    29098089  ...            18.579081             0.830798
2      G26/Neurology    28729901  ...             4.514742             0.981974
3      G26/Neurology    29074035  ...             6.114408             0.981470
4      G26/Neurology    29050280  ...            43.508338             0.979416
...              ...         ...  ...                  ...                  ...
20625  G26/Neurology    29327876  ...             5.986113             0.976717
20626  G26/Neurology    29215476  ...             5.986113             0.976717
20627  G26/Neurology    29212467  ...             5.986113             0.976717
20628  G26/Neurology    30179101  ...             6.690498             0.976247
20629  G26/Neurology    29704435  ...           

In [ ]:
uploaded = files.upload()

Saving Book1.xlsx to Book1.xlsx


In [ ]:
import pandas as pd
cite_score = pd.read_excel(io.BytesIO(uploaded['Book1.xlsx']))

print(cite_score.head)

ImportError: ignored

In [ ]:
uploaded = files.upload()
fi = pd.read_csv(io.BytesIO(uploaded['feature_importance.csv']))
print(fi.head)

Saving feature_importance.csv to feature_importance.csv
<bound method NDFrame.head of       ARTICLE_ID  ...       CTR
0       27356366  ...  0.100000
1       27864009  ...  0.000000
2       27946189  ...  0.000000
3       26254810  ...  0.000000
4       29729161  ...  0.053333
...          ...  ...       ...
4311    27941463  ...  0.000000
4312    27328582  ...  0.071429
4313    24209811  ...  0.000000
4314    26665441  ...  0.000000
4315    25691001  ...  0.068966

[4316 rows x 15 columns]>


In [ ]:
uploaded = files.upload()
fi = pd.read_csv(io.BytesIO(uploaded['asjc_code.csv']))
print(fi.head)

In [ ]:
#list data types of candidate features
# some need more work than others. Lets start with the features that we most easily can turn into ranks
fi.dtypes

ARTICLE_ID               int64
JOURNAL                   bool
WORD_COUNT             float64
MEDICAL_RELEVANCE      float64
MEDICAL_DENSITY        float64
AVG_MEDICAL_DENSITY    float64
ALTMETRIC_SCORE        float64
AVG_ALTMETRIC_SCORE    float64
SOURCE_DISPLAY_NAME     object
SOURCE_DOMAIN           object
SJR                    float64
H_INDEX                float64
ALEXA_RANK               int64
DIFFBOT_LABELS          object
CTR                    float64
dtype: object

In [ ]:
journal_ranking = fi[['SOURCE_DISPLAY_NAME','SJR','H_INDEX','ALEXA_RANK']]

In [ ]:
journal_ranking.SOURCE_DISPLAY_NAME.nunique()/journal_ranking.SOURCE_DISPLAY_NAME.count()

0.13183503243744207

Applying the MC4 algo to a sample data set to explore how the ranking works in practice.

In [19]:
#Import test data set and apply rank aggregator to better understand output structure and options
from mc4.algorithm import mc4_aggregator
url ='https://raw.githubusercontent.com/kalyaniuniversity/MC4/master/test_datasets/test_dataset_1.csv'
mc4_test_data = pd.read_csv(url,error_bad_lines=False, header=0,index_col=0)

test_data = mc4_test_data[['T-Test','Ranksum','WT-Test','F-Test','KS-Test']]

aggregated_ranks = mc4_aggregator(test_data, 
                                  header_row = 0, 
                                  index_col = 0,
                                  algo= 'mct',
                                  iterations=300,
                                  erg_number = 0.15
                                  ) 

ranks = pd.DataFrame.from_dict(aggregated_ranks, orient='index',columns=['mc_rank'])

test_data.merge(ranks, how='inner',left_index=True,right_index=True)

,T-Test,Ranksum,WT-Test,F-Test,KS-Test,mc_rank
item_0,84,93,84,84,85,87
item_1,65,54,65,65,19,63
item_2,3,3,3,3,1,2
item_3,58,64,58,58,20,57
item_4,29,38,29,29,56,33
...,...,...,...,...,...,...
item_95,9,13,9,9,54,12
item_96,64,87,64,64,99,70
item_97,53,61,53,53,83,58
item_98,57,63,57,57,55,61
